<h1><center>Centro Universitário IESB</center></h1>
Curso: Ciências de Dados - Põs-Graduação<br>
Aluno: ROBSON BATISTA DA SILVA<br>
Disciplina: Data Mining e Machine Learning II

   

# Importando as bibliotecas

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# <center>Dicionário de Dados</center>


**BAD:** 1 = cliente inadimplente no empréstimo 0 = empréstimo reembolsado (variável target)

**LOAN**: Valor do empréstimo

**MORTDUE**: Valor devido da hipoteca existente

**VALUE**: valor da propriedade atual

**REASON**: DebtCon = consolidação da dívida; HomeImp = melhoramento da casa

**JOB**: Seis categorias profissionais

**YOJ**: Anos no emprego atual

**DEROG**: Número de principais relatórios depreciativos

**DELINQ**: número de linhas de crédito inadimplentes

**CLAGE**: Idade da linha comercial mais antiga em meses

**NINQ**: Número de linhas de crédito recentes

**CLNO**: Número de linhas de crédito

**DEBTINC**: Rácio dívida / rendimento

* ### Carregando os dados

In [ ]:
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df.head()

* # Verificando os tipos e os valores nulos

In [ ]:
df.info()

O Dataset possui 5.960 linhas e 13 colunas,com exceção das variáveis BAD e LOAN todas as outras variáveis possui valores missing e duas variáveis object JOB e REASON

# <center>**Exploração dos Dados**</center>

In [ ]:
#Estatísticas Descritivas
df.describe()

Estatísticas de todas variáveis quantitativas

In [ ]:
# Verificando a distribuição da variável BAD(Target)
df['BAD'].plot.hist(bins=5)

In [ ]:
#frenquência dos dados da variável BAD
df['BAD'].value_counts()

Como podemos observar no gráfico acima e na frequência, a distribuição da variável Target(BAD) não está equilibrada, provavelmente as pessoas que pagaram vão enviesar o modelo.

# <center>Tratamento dos Dados</center>

In [ ]:
#Verificando a quantidade de valores missing nas variáveis
df.isnull().sum()

Ao analisar a quantidade de missing na base, temos duas opções: imputação de dados ou a remoção dos valores com missing, nesse caso eu vou remover os valores missing.

In [ ]:
# Removendo os valores missing value da base
#df2 =df.copy()
df.dropna(axis=0,how='any',inplace= True)
df.info(), df.isna().any() 


In [ ]:
#valores da variável target
df['BAD'].value_counts().plot(kind='bar',title='Frequência da Variável BAD')
df['BAD'].value_counts()

Ao remover os missing da base a variável target, continuou muito desbalanceada 

### Análise Descritiva Exploratória



A avaliação dos histogramas mostra inicialmente que :
- A variável BAD (Target) possui poucos valores 1 para treinamento do modelo
- A maior parte dos valores totais de financiamento (LOAN) possuem uma distribuição próxima da normalidade, e os valores a receber(MORTDUE), na média, são maiores que os totais emprestados. Observa-se o terror dos juros bancários
- Os valores das propriedades possuem distribuição próxima dos valores dos financiamentos
- O DEROG, algo equivalente à um aviso de negativação do serviço de proteção ao consumidor, é baixo, contudo possui uma correlação próxima de moderada(p=0,25) com os maus pagadores.
- O DELINQ, linhas de crédito com inadimplência, também possui correlação próxima à moderada(p=0,27) com maus pagadores
- O número de linhas de crédito possui correlação, mas a intensidade é menor(p=0,13), com maus pagadores
- Por fim, a base possui um indicador (Débitos/Renda) que possui uma correlação próxima a moderada (p=0,23), sendo um bom indicador.

In [ ]:
# Correlação das variáveis numéricas
plt.figure(figsize= (15, 8))
sns.heatmap(df.corr(), square=True, annot=True, linewidth=0.5)

Ao verificar a análise de correlação, observa-se que as variáveis VALUE X MORTDUE tem a maior correlação.

In [ ]:
#Distribuição da variável REASON por BAD
df.groupby(['BAD'])['REASON'].value_counts()

A variável razão do débito apresentou na categoria Debtcon a maior quatidade de não pagar, mas por um outro lado essa categoria é aque mais quinta seu debidos.

In [ ]:
#Distribuição da variável JOB por BAD
df.groupby(['BAD'])['JOB'].value_counts()

Observa-se que o grupo que trabalho Other e ProfExe possuem um número maior de maus pagadores

### Featuring Engineering

Para melhor ajuste ao modelo, foram dummizadas as tabelas com type Object

In [ ]:
# Gerando Dummies para modelos que utilizam apenas variaveis numéricas

df = pd.get_dummies(df, columns=['REASON', 'JOB'])
df.head().T

In [ ]:
#Normalizando os dados

#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#df = pd.DataFrame(sc.fit_transform(df), columns=df.columns)

## Geração Amostras de Treino e Teste

Nota: Neste trabalho foi realizada a modelagem utilzando uma amostra para validação inclusive, contudo, devida a baixa quantidade de registros, foi utilizado apenas treino e teste 

In [ ]:
# importando a biblioteca
from sklearn.model_selection import train_test_split

#Separando em treino e teste
treino, teste = train_test_split(df, test_size=0.20, random_state=42)

# Não vou usar o dataset de validação

treino.shape, teste.shape  

In [ ]:
# Lista das colunas que serão usadas
usadas = [c for c in treino.columns if c not in ['BAD','REASON','JOB']]

In [ ]:
#importando métrica
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score


## Modelo RandomForest Classifier

In [ ]:
# importanto o modelo
from sklearn.ensemble import RandomForestClassifier

#instanciando o modelo
rf = RandomForestClassifier(n_estimators=200,random_state=42)

In [ ]:
# treinando o modelo
rf.fit(treino[usadas], treino['BAD'])

# gerando predicoes do modelo com os dados de teste
pred_teste = rf.predict(teste[usadas])

#Medindo a acuracia nos dados de teste
accuracy_score(teste['BAD'],pred_teste), balanced_accuracy_score(teste['BAD'],pred_teste), f1_score(teste['BAD'],pred_teste)

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index=usadas).sort_values().plot.barh()

No gráfico acima observa-se que as variáveis que foram geradas dummers, não são importantes para o modelo

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_teste)


A Matriz de Confusão mostra que a base de dados é bastante desbalanceada, Assim, a análise das métricas de especifidade e esforço pode realçar os falsos positivos

### Utilizando o RandonForest Classifier com ajuste nos parâmetros


Foram ajustados os parâmetros de aumentando o número de estimadores para 500, quando o default é 100, e informando que o número de folhas aceitavel para as ramificações das árvores de decisão como 2.

In [ ]:
# Setando parametros
rf2 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=-1, n_estimators=500,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,  class_weight='balanced')
# treinando o modelo RF2
rf2.fit(treino[usadas], treino['BAD'])

In [ ]:
#relizando a predicao do RF2 com base teste
pred_teste2 = rf2.predict(teste[usadas])

#métrica para RF2 validacao
accuracy_score(teste['BAD'],pred_teste2), balanced_accuracy_score(teste['BAD'],pred_teste2), f1_score(teste['BAD'],pred_teste2)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_teste2)

Após o ajustes no modelo, deve um pequena melhora, mas nada muito significativos.

## Modelo XGBoost

In [ ]:
# Importar o modelo
from xgboost import XGBClassifier

# Instanciar o modelo
xgb = XGBClassifier(n_estimators=900, n_jobs=-1, random_state=42, learning_rate=0.05)

# treinando o modelo
xgb.fit(treino[usadas],treino['BAD']) 



In [ ]:
# Fazendo predições
pred_xgb_teste = xgb.predict(teste[usadas])

# Metrícas XGB teste
accuracy_score(teste['BAD'],pred_xgb_teste), balanced_accuracy_score(teste['BAD'],pred_xgb_teste), f1_score(teste['BAD'],pred_xgb_teste)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_xgb_teste)

O XGBoost apresentou uma melhora um pouco maior no modelo, contudo não foi significante.

# Conclusão

Ao comparar os modelos pela a acurácia e o F1 score, o modelo que teve o melhor desenpenho foi o
XGBoost, ele obteve 0,96 de acuracia e 0,70 de F1 score, o modelo RandonForest Classifier sem ajustes  0.958 e o f1 score foi de 0,66, já o modelo RandomForest Classifier com ajustes  obteve a acuracia de 0.956 e o F1 Score 0,68.

Todos os modelos se sairam bem, mas quando olhamos as matriz de confusão, percebemos que os modelos não são bons para prever se o cliente estão inadimplente , mas é otimo para prever se em dia, os modelos tem muitas falso positivos nesse caso, 

Portanto teria que fazer um novo tratamento nos dados e ou uma imputação  missing values ou até normalizar os dados, para que seja rodado um novo modelo. 